In [ ]:
#! /usr/bin/env python

import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
# FLAGS._parse_flags()
# print("\nParameters:")
# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr.upper(), value))
# print("")

def preprocess():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

def main(argv=None):
    x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)

if __name__ == '__main__':
    tf.app.run()

Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 18758
Train/Dev split: 9596/1066
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using con

2019-06-29T16:22:27.110684: step 59, loss 1.68814, acc 0.546875
2019-06-29T16:22:27.418463: step 60, loss 1.52971, acc 0.546875
2019-06-29T16:22:27.678130: step 61, loss 1.9034, acc 0.4375
2019-06-29T16:22:27.945461: step 62, loss 1.7403, acc 0.453125
2019-06-29T16:22:28.245964: step 63, loss 1.95955, acc 0.4375
2019-06-29T16:22:28.527128: step 64, loss 1.56371, acc 0.546875
2019-06-29T16:22:28.794866: step 65, loss 1.8479, acc 0.484375
2019-06-29T16:22:29.078728: step 66, loss 1.82928, acc 0.484375
2019-06-29T16:22:29.360861: step 67, loss 2.09852, acc 0.46875
2019-06-29T16:22:29.633434: step 68, loss 1.52135, acc 0.640625
2019-06-29T16:22:29.909255: step 69, loss 1.47703, acc 0.609375
2019-06-29T16:22:30.208958: step 70, loss 1.3347, acc 0.578125
2019-06-29T16:22:30.492395: step 71, loss 1.45748, acc 0.546875
2019-06-29T16:22:30.762065: step 72, loss 1.51223, acc 0.5
2019-06-29T16:22:31.044005: step 73, loss 1.9417, acc 0.421875
2019-06-29T16:22:31.326762: step 74, loss 1.57385, acc 

2019-06-29T16:23:06.841079: step 185, loss 0.967461, acc 0.578125
2019-06-29T16:23:07.128346: step 186, loss 0.892052, acc 0.625
2019-06-29T16:23:07.425225: step 187, loss 1.14942, acc 0.53125
2019-06-29T16:23:07.706480: step 188, loss 1.08552, acc 0.5
2019-06-29T16:23:07.962037: step 189, loss 0.945184, acc 0.5625
2019-06-29T16:23:08.243291: step 190, loss 0.889716, acc 0.625
2019-06-29T16:23:08.508920: step 191, loss 0.944595, acc 0.609375
2019-06-29T16:23:08.774549: step 192, loss 1.23426, acc 0.609375
2019-06-29T16:23:09.045307: step 193, loss 0.858688, acc 0.671875
2019-06-29T16:23:09.310936: step 194, loss 0.898139, acc 0.625
2019-06-29T16:23:09.576565: step 195, loss 0.797731, acc 0.71875
2019-06-29T16:23:09.842195: step 196, loss 0.761984, acc 0.703125
2019-06-29T16:23:10.140257: step 197, loss 1.26495, acc 0.515625
2019-06-29T16:23:10.421510: step 198, loss 0.851109, acc 0.71875
2019-06-29T16:23:10.687138: step 199, loss 0.951732, acc 0.546875
2019-06-29T16:23:10.958281: step 

2019-06-29T16:23:43.710058: step 305, loss 0.950459, acc 0.53125
2019-06-29T16:23:43.981537: step 306, loss 0.846425, acc 0.625
2019-06-29T16:23:44.257856: step 307, loss 0.662081, acc 0.640625
2019-06-29T16:23:44.523484: step 308, loss 0.642837, acc 0.640625
2019-06-29T16:23:44.773488: step 309, loss 0.752559, acc 0.6875
2019-06-29T16:23:45.044447: step 310, loss 0.728639, acc 0.625
2019-06-29T16:23:45.341327: step 311, loss 0.761337, acc 0.609375
2019-06-29T16:23:45.614473: step 312, loss 0.532364, acc 0.8125
2019-06-29T16:23:45.871420: step 313, loss 0.857393, acc 0.578125
2019-06-29T16:23:46.137050: step 314, loss 0.589154, acc 0.6875
2019-06-29T16:23:46.418306: step 315, loss 0.722751, acc 0.625
2019-06-29T16:23:46.715184: step 316, loss 0.631547, acc 0.65625
2019-06-29T16:23:47.019067: step 317, loss 0.424247, acc 0.78125
2019-06-29T16:23:47.300320: step 318, loss 0.745386, acc 0.609375
2019-06-29T16:23:47.605159: step 319, loss 0.741354, acc 0.546875
2019-06-29T16:23:47.908827: 

2019-06-29T16:24:25.517759: step 428, loss 0.782794, acc 0.484375
2019-06-29T16:24:25.793955: step 429, loss 0.656094, acc 0.703125
2019-06-29T16:24:26.077156: step 430, loss 0.591494, acc 0.71875
2019-06-29T16:24:26.354354: step 431, loss 0.733316, acc 0.65625
2019-06-29T16:24:26.639557: step 432, loss 0.597744, acc 0.671875
2019-06-29T16:24:26.936768: step 433, loss 0.473366, acc 0.734375
2019-06-29T16:24:27.207963: step 434, loss 0.660236, acc 0.6875
2019-06-29T16:24:27.475152: step 435, loss 0.668944, acc 0.640625
2019-06-29T16:24:27.745344: step 436, loss 0.625865, acc 0.65625
2019-06-29T16:24:28.013535: step 437, loss 0.629141, acc 0.765625
2019-06-29T16:24:28.282727: step 438, loss 0.627616, acc 0.6875
2019-06-29T16:24:28.551918: step 439, loss 0.738822, acc 0.578125
2019-06-29T16:24:28.819108: step 440, loss 0.666156, acc 0.609375
2019-06-29T16:24:29.084298: step 441, loss 0.674604, acc 0.6875
2019-06-29T16:24:29.384511: step 442, loss 0.647496, acc 0.65625
2019-06-29T16:24:29.

...

2019-06-29T16:41:18.588999: step 3744, loss 0.00265725, acc 1
2019-06-29T16:41:18.855188: step 3745, loss 0.0020126, acc 1
2019-06-29T16:41:19.131385: step 3746, loss 0.00251971, acc 1
2019-06-29T16:41:19.403579: step 3747, loss 0.00264404, acc 1
2019-06-29T16:41:19.677774: step 3748, loss 0.00947598, acc 1
2019-06-29T16:41:19.948968: step 3749, loss 0.00658432, acc 1
2019-06-29T16:41:20.207151: step 3750, loss 0.0134069, acc 1
2019-06-29T16:41:20.484349: step 3751, loss 0.00346213, acc 1
2019-06-29T16:41:20.773556: step 3752, loss 0.00804606, acc 1
2019-06-29T16:41:21.053753: step 3753, loss 0.00272885, acc 1
2019-06-29T16:41:21.388992: step 3754, loss 0.00170025, acc 1
2019-06-29T16:41:21.683202: step 3755, loss 0.0111747, acc 1
2019-06-29T16:41:21.963401: step 3756, loss 0.00118243, acc 1
2019-06-29T16:41:22.234627: step 3757, loss 0.00259779, acc 1
2019-06-29T16:41:22.513825: step 3758, loss 0.00325835, acc 1
2019-06-29T16:41:22.839057: step 3759, loss 0.00788098, acc 1
2019-06-29T

2019-06-29T16:41:56.545092: step 3872, loss 0.00466702, acc 1
2019-06-29T16:41:56.829295: step 3873, loss 0.00451504, acc 1
2019-06-29T16:41:57.128508: step 3874, loss 0.0310061, acc 0.984375
2019-06-29T16:41:57.410709: step 3875, loss 0.00852509, acc 1
2019-06-29T16:41:57.696912: step 3876, loss 0.0420798, acc 0.984375
2019-06-29T16:41:57.980114: step 3877, loss 0.0019676, acc 1
2019-06-29T16:41:58.267319: step 3878, loss 0.00111441, acc 1
2019-06-29T16:41:58.548519: step 3879, loss 0.00677224, acc 1
2019-06-29T16:41:58.831721: step 3880, loss 0.00969407, acc 1
2019-06-29T16:41:59.117925: step 3881, loss 0.0117361, acc 1
2019-06-29T16:41:59.411133: step 3882, loss 0.0113912, acc 1
2019-06-29T16:41:59.707344: step 3883, loss 0.00723008, acc 1
2019-06-29T16:41:59.992547: step 3884, loss 0.00089314, acc 1
2019-06-29T16:42:00.283285: step 3885, loss 0.0016729, acc 1
2019-06-29T16:42:00.572490: step 3886, loss 0.0108487, acc 1
2019-06-29T16:42:00.857692: step 3887, loss 0.00665007, acc 1
2

2019-06-29T16:42:35.341836: step 4000, loss 1.27148, acc 0.722326

Saved model checkpoint to C:\Users\mao\Desktop\Tensorflow\CNN\1\cnn-text-classification-tf-master\runs\1561796526\checkpoints\model-4000

2019-06-29T16:42:36.432610: step 4001, loss 0.00121615, acc 1
2019-06-29T16:42:36.970992: step 4002, loss 0.00188656, acc 1
2019-06-29T16:42:37.245187: step 4003, loss 0.00339027, acc 1
2019-06-29T16:42:37.511377: step 4004, loss 0.0043059, acc 1
2019-06-29T16:42:37.776566: step 4005, loss 0.00608671, acc 1
2019-06-29T16:42:38.057766: step 4006, loss 0.00385668, acc 1
2019-06-29T16:42:38.320954: step 4007, loss 0.00117497, acc 1
2019-06-29T16:42:38.591146: step 4008, loss 0.0515545, acc 0.96875
2019-06-29T16:42:38.859337: step 4009, loss 0.0204826, acc 0.984375
2019-06-29T16:42:39.145540: step 4010, loss 0.00384494, acc 1
2019-06-29T16:42:39.413732: step 4011, loss 0.00111465, acc 1
2019-06-29T16:42:39.680922: step 4012, loss 0.00611243, acc 1
2019-06-29T16:42:39.950113: step 4013, lo

2019-06-29T16:43:13.705980: step 4125, loss 0.0054975, acc 1
2019-06-29T16:43:13.991234: step 4126, loss 0.00459823, acc 1
2019-06-29T16:43:14.263430: step 4127, loss 0.00407853, acc 1
2019-06-29T16:43:14.551633: step 4128, loss 0.00913581, acc 1
2019-06-29T16:43:14.849846: step 4129, loss 0.00157687, acc 1
2019-06-29T16:43:15.151061: step 4130, loss 0.00119147, acc 1
2019-06-29T16:43:15.450273: step 4131, loss 0.017586, acc 0.984375
2019-06-29T16:43:15.726871: step 4132, loss 0.00144997, acc 1
2019-06-29T16:43:16.015076: step 4133, loss 0.0106403, acc 1
2019-06-29T16:43:16.299277: step 4134, loss 0.0198985, acc 1
2019-06-29T16:43:16.600491: step 4135, loss 0.000897891, acc 1
2019-06-29T16:43:16.887696: step 4136, loss 0.0309875, acc 0.984375
2019-06-29T16:43:17.172899: step 4137, loss 0.0021771, acc 1
2019-06-29T16:43:17.457101: step 4138, loss 0.0478239, acc 0.984375
2019-06-29T16:43:17.789339: step 4139, loss 0.00140158, acc 1
2019-06-29T16:43:18.073540: step 4140, loss 0.000758136,

2019-06-29T16:43:52.770671: step 4252, loss 0.00312327, acc 1
2019-06-29T16:43:53.132930: step 4253, loss 0.0084107, acc 1
2019-06-29T16:43:54.052585: step 4254, loss 0.00158817, acc 1
2019-06-29T16:43:54.423848: step 4255, loss 0.00934736, acc 1
2019-06-29T16:43:54.723061: step 4256, loss 0.00167259, acc 1
2019-06-29T16:43:55.005262: step 4257, loss 0.00230418, acc 1
2019-06-29T16:43:55.287464: step 4258, loss 0.00204756, acc 1
2019-06-29T16:43:55.574668: step 4259, loss 0.000723854, acc 1
2019-06-29T16:43:55.855868: step 4260, loss 0.0303083, acc 0.984375
2019-06-29T16:43:56.152079: step 4261, loss 0.00221511, acc 1
2019-06-29T16:43:56.670449: step 4262, loss 0.00190501, acc 1
2019-06-29T16:43:56.971662: step 4263, loss 0.00168531, acc 1
2019-06-29T16:43:57.258866: step 4264, loss 0.00100231, acc 1
2019-06-29T16:43:57.554076: step 4265, loss 0.00318897, acc 1
2019-06-29T16:43:57.840281: step 4266, loss 0.00366666, acc 1
2019-06-29T16:43:58.121480: step 4267, loss 0.0117894, acc 1
201